# Assignment 3. Pruning for LLM

## Goals

본 실습에서는 대규모 언어 모델(Large Language Model, LLM)의 크기를 효과적으로 줄이는 Pruning 기법을 학습합니다. 특히 Magnitude-based pruning과 최근 주목받는 Wanda 기법을 활용하여, 파라미터의 수를 효율적으로 감소시키면서 모델의 성능을 유지하는 방법을 실습합니다.


## Contents

1. **Magnitude-based Pruning 실습**:
   - 간단한 magnitude 기반 pruning 방법을 통해 모델 크기를 감소시키고 성능 변화를 확인합니다.
2. **Wanda를 이용한 Pruning 실습**:
   - Activation의 중요도를 측정하여 더 정교하게 pruning을 수행하는 Wanda 방법을 구현하고 모델의 성능을 비교합니다.

## Setup

실습에 필요한 패키지를 설치합니다.

In [ ]:
print('Installing packages...')
!pip install torch transformers==4.31.0 accelerate==0.21.0 sentencepiece==0.1.99 tokenizers==0.13.3 datasets==2.15.0 tqdm zstandard

필요한 모듈을 불러옵니다.

In [ ]:
import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from functools import partial
import gc

### 모델 평가

Wikitext-2 데이터셋을 사용하여 모델의 성능을 평가하는 지표인 perplexity를 계산합니다.

**Perplexity란?**
- Perplexity는 언어 모델이 주어진 텍스트를 얼마나 잘 예측하는지를 수치로 나타낸 지표입니다.
- 수학적으로는 모델이 예측한 확률분포의 "불확실성"을 측정하는 값이며, 값이 **낮을수록 모델의 성능이 좋다**고 해석합니다.
- 단어 $\{w_1, w_2, w_3, ..., w_N\}$으로 구성된 문장의 Perplexity는 다음과 같은 수식으로 나타낼 수 있습니다.
    - $Perplexity = \sqrt[N]{\frac{1}{\prod_{i=1}^{N} P(w_i | w_1, w_2, ..., w_{i-1})}}$
    - 여기서 $P(w_i | w_1, w_2, ..., w_{i-1})$은 $i$번째에 $w_i$ 단어를 생성할 확률을 의미합니다.
    - 즉, Perplexity는 문자의 발생 확률에 대한 역수를 의미하게 됩니다.
- 직관적으로 Perplexity가 10이라면 모델이 다음 단어 후보를 10개 정도로 생각하고 있다고 볼 수 있습니다.

In [13]:
def evaluate(model, tokenizer):
    """
    모델의 perplexity를 계산하는 함수입니다.
    
    Args:
        model: 평가할 모델
        tokenizer: 토크나이저
        
    Returns:
        float: 계산된 perplexity 값
    """
    # 테스트 데이터셋 로드 및 전처리
    testenc = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
    testenc = tokenizer("\n\n".join(testenc['text']), return_tensors='pt')
    
    # 입력 데이터를 모델 디바이스로 이동
    testenc = testenc.input_ids.to(model.device)
    nsamples = 40
    model = model.eval()  # 평가 모드로 설정

    # Negative log likelihood 계산
    nlls = []
    for i in tqdm.tqdm(range(nsamples), desc="evaluating..."):
        # 배치 데이터 준비
        batch = testenc[:, (i * 2048):((i + 1) * 2048)].to(model.device)
        
        # 모델 추론
        with torch.no_grad():
            lm_logits = model(batch).logits
            
        # 로짓과 레이블 시프트
        shift_logits = lm_logits[:, :-1, :].contiguous().float()
        shift_labels = testenc[:, (i * 2048):((i + 1) * 2048)][:, 1:]
        
        # 손실 계산
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        neg_log_likelihood = loss.float() * 2048
        nlls.append(neg_log_likelihood)

    # Perplexity 계산 및 반환
    return torch.exp(torch.stack(nlls).sum() / (nsamples * 2048))

### OPT 1.3B 모델 로딩

OPT-1.3B 모델을 로딩하고 평가합니다.
여기서 tokenizer는 텍스트(문장)를 모델이 이해할 수 있는 작은 단위(token)로 나누는 역할을 수행합니다.

In [ ]:
model_path = "facebook/opt-1.3b"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

# Evaluate the model
model_perplexity = evaluate(model, tokenizer)
print(f"\nmodel perplexity: {model_perplexity:.2f}")

## [실습 1] Magnitude-based Pruning 구현

Magnitude-based pruning 함수를 구현하세요. 구현은 CNN에서 magnitude-based pruning을 구현하는 방식과 유사합니다.

In [15]:
@torch.no_grad()
def prune_magnitude_opt(model, sparsity):
    for n, m in model.named_modules():
        if isinstance(m, nn.Linear) and "lm_head" not in n:
            W = m.weight.data
            ##################### YOUR CODE STARTS HERE #####################
            num_elements = W.numel()
            num_zeros = round(num_elements * sparsity)
            importance = torch.abs(W)
            threshold = torch.kthvalue(importance.flatten(), num_zeros)[0]
            mask = importance > threshold
            ##################### YOUR CODE ENDS HERE #######################
            W.mul_(mask)

Pruning된 모델을 평가합니다.

In [ ]:
del model
gc.collect()
torch.cuda.empty_cache()
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
prune_magnitude_opt(model, 0.5)

# Evaluate the model
model_perplexity = evaluate(model, tokenizer)
print(f"\nmodel perplexity (magnitude 50%): {model_perplexity:.2f}")

## [실습 2] Calibration 데이터셋 준비
 
[Wanda](https://arxiv.org/pdf/2306.11695)를 사용하여 importance를 계산하기 위해서는 calibration 데이터셋을 통해 activation을 추출해야 합니다.

아래 빈칸을 작성하여 `activation_norm`을 계산하세요.
Wanda 기법에서는 `activation_norm`을 L2 norm으로 계산하지만, 구현에서는 calibration 데이터셋에 대해 여러 번 반복하여 누적하므로 
`activation_norm` 계산 시 제곱을 적용한 형태로 누적합니다. (이후 값을 사용할 때는 제곱근을 적용해야 합니다.)

In [17]:
def get_calib_dataset(tokenizer=None, n_samples=256, block_size=512):
    dataset = load_dataset("mit-han-lab/pile-val-backup", split="validation")
    dataset = dataset.shuffle(seed=42)
    samples = []
    n_run = 0
    for data in dataset:
        line = data["text"]
        line = line.strip()
        line_encoded = tokenizer.encode(line)
        if len(line_encoded) > block_size:
            continue
        sample = torch.tensor([line_encoded])
        if sample.numel() == 0:
            continue
        samples.append(sample)
        n_run += 1
        if n_run == n_samples:
            break

    # now concatenate all samples and split according to block size
    cat_samples = torch.cat(samples, dim=1)
    n_split = cat_samples.shape[1] // block_size
    print(f" * Split into {n_split} blocks")
    return [cat_samples[:, i*block_size:(i+1)*block_size] for i in range(n_split)]

@torch.no_grad()
def get_calib_feat(model, tokenizer):
    input_dict = dict()
    nsamples_dict = dict()
    def add_batch(m, x, y, name):
        if name not in input_dict:
            input_dict[name] = torch.zeros((m.weight.data.shape[1]), device=m.weight.data.device)
            nsamples_dict[name] = 0

        if isinstance(x, tuple):
            x = x[0]

        if len(x.shape) == 2:
            x = x.unsqueeze(0)
        tmp = x.shape[0]
        if len(x.shape) == 3:
            x = x.reshape((-1, x.shape[-1]))
        x = x.t()

        input_dict[name] *= nsamples_dict[name] / (nsamples_dict[name] + tmp)
        nsamples_dict[name] += tmp

        x = x.type(torch.float32)
        ##################### YOUR CODE STARTS HERE #####################
        # activation_norm을 계산하세요.
        # x.shape => (hidden_size, batch_size)
        activation_norm = torch.norm(x, p=2, dim=1) ** 2
        # activation_norm.shape => (hidden_size)
        ##################### YOUR CODE ENDS HERE #######################
        input_dict[name] += activation_norm / nsamples_dict[name]

    hooks = []
    for name, m in model.named_modules():
        if isinstance(m, nn.Linear) and "lm_head" not in name:
            hooks.append(
                m.register_forward_hook(
                    partial(add_batch, name=name)))

    print("Collecting norm of input activations...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    samples = get_calib_dataset(tokenizer)
    pbar = tqdm.tqdm(samples)
    for input_ids in pbar:
        input_ids = input_ids.to(device)
        model(input_ids)

    for hook in hooks:
        hook.remove()
    return input_dict

In [ ]:
del model
gc.collect()
torch.cuda.empty_cache()
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
input_feat = get_calib_feat(model, tokenizer)

## [실습 3] Wanda Pruning 구현
 
Wanda 논문을 참고하여 아래 빈 칸을 채워 Wanda Pruning을 구현하고 실행해보세요.

단, 원래 Wanda에서는 weight parameter의 행(row)별로 동일한 희소성(sparsity)으로 pruning하지만, 
본 실습에서는 이를 고려하지 않고 전체 weight에 대해 동일한 희소성을 적용하겠습니다.

In [19]:
@torch.no_grad()
def prune_wanda_opt(model, sparsity, input_feat):
    for n, m in model.named_modules():
        if isinstance(m, nn.Linear) and "lm_head" not in n:
            W = m.weight.data
            ##################### YOUR CODE STARTS HERE #####################
            num_elements = W.numel()
            num_zeros = round(num_elements * sparsity)
            importance = torch.abs(W) * torch.sqrt(input_feat[n])
            threshold = torch.kthvalue(importance.flatten(), num_zeros)[0]
            mask = importance > threshold
            ##################### YOUR CODE ENDS HERE #######################
            W.mul_(mask)

In [ ]:
del model
gc.collect()
torch.cuda.empty_cache()
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
prune_wanda_opt(model, sparsity=0.5, input_feat=input_feat)

# Evaluate the model
model_perplexity = evaluate(model, tokenizer)
print(f"\nmodel perplexity (wanda 50%): {model_perplexity:.2f}")